# Contact Lenses Supply Chain Optimization

## Modeling the Supply Chain

### Objective Function

Minimize the total operating costs, which include:
  - **Production Costs**: Cost of producing units at each manufacturing site.
  - **Transportation Costs**: Cost of shipping products from manufacturing sites to distribution centers and warehouses.

Decision Variables
  - **Production Quantities**: Units of each product produced at each manufacturing site.
  - **Transportation Quantities**: Units of each product shipped from manufacturing sites to distribution centers and warehouses.

Constraints
  1. **Production Capacity Constraints**: Each manufacturing site cannot produce more than its capacity.
  2. **Demand Fulfillment Constraints**: Customer demand in each country must be met.
  3. **Non-negativity Constraints**: Decision variables must be non-negative.

## Implementing the Model

### Import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
from pulp import *

### Load the Data

In [3]:
products = pd.read_excel('Products.xlsx')
production = pd.read_excel('Production.xlsx')
demand = pd.read_excel('Demand.xlsx')
distribution = pd.read_excel('Distribution.xlsx')

### Prepare the Data

**Products Data**

In [8]:
# Clean and process products data (remove the 'USD' string and convert to float)
products['cost_unit_production'] = products['cost_unit_production'].astype(str).str.replace(' USD', '').astype(float)


**Production Data**

Create a list of production capacities.

In [9]:
# Production capacities
production_sites = production['Site Code'].tolist()
production_capacities = dict(zip(production['Site Code'], production['Unit Production']))

**Demand Data**

In [ ]:
# Set index to Country_Code
demand.set_index('Country_Code', inplace=True)

# Demand for each product in each country
demand = demand.stack().reset_index()
demand.columns = ['Country_Code', 'Product_Code', 'Demand']

**Distribution Centers**

In [12]:
distribution_centers = distribution['Country_code'].unique().tolist()

### Define the Optimization Model

In [15]:
# Create the LP Minimization problem
prob = LpProblem("Contact_Lenses_Supply_Chain_Optimization", LpMinimize)

### Define Decision Variables

**Variables for Production Quantities**

In [16]:
# Decision variables for production quantities
production_vars = LpVariable.dicts("Production",
                                   ((site, product) for site in production_sites for product in products['Code']),
                                   lowBound=0,
                                   cat='Continuous')

**Variables for Transportation Quantities**

In [17]:
# Decision variables for transportation quantities
transport_vars = LpVariable.dicts("Transport",
                                  ((site, country, product) for site in production_sites
                                   for country in demand['Country_Code'].unique()
                                   for product in products['Code']),
                                  lowBound=0,
                                  cat='Continuous')

### Define the Objective Function

We aim to minimize the total cost, which includes production and transportation costs.

In [20]:
# Production costs
production_costs = lpSum([production_vars[(site, product)] * products.loc[products['Code'] == product, 'cost_unit_production'].values[0]
                          for site in production_sites for product in products['Code']])

# Transportation costs
# For simplicity, assume a unit transportation cost (this can be replaced with actual costs if available)
unit_transport_cost = 0.1  # USD per unit per km

# Calculate distances between production sites and countries (requires coordinates)
# For this example, we'll assume a fixed cost
transportation_costs = lpSum([transport_vars[(site, country, product)] * unit_transport_cost
                              for site in production_sites
                              for country in demand['Country_Code'].unique()
                              for product in products['Code']])

# Total cost
total_cost = production_costs + transportation_costs

# Set the objective
prob += total_cost

### Define Constraints

**Production Capacity Constraints**

In [21]:
for site in production_sites:
    prob += lpSum([production_vars[(site, product)] for product in products['Code']]) <= production_capacities[site], f"Capacity_{site}"

**Demand Fulfillment Constraints**

In [22]:
for index, row in demand.iterrows():
    country = row['Country_Code']
    product = row['Product_Code']
    demand_qty = row['Demand']
    prob += lpSum([transport_vars[(site, country, product)] for site in production_sites]) >= demand_qty, f"Demand_{country}_{product}"

**Link Between Production and Transportation**

We need to ensure that the amount transported from a site does not exceed its production.

In [23]:
for site in production_sites:
    for product in products['Code']:
        prob += lpSum([transport_vars[(site, country, product)] for country in demand['Country_Code'].unique()]) <= production_vars[(site, product)], f"Transport_{site}_{product}"

### Solve the Model

In [24]:
# Solve the optimization problem
prob.solve()

# Print the status
print(f"Status: {LpStatus[prob.status]}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mateuszpolis/Documents/University/Semester 5/Mathematical Optimization Algorithms/.conda/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8l/df7brwpx0vj8qwl22c82bj5w0000gn/T/b7ada75ec03746a6a7a778cc1a16c0ea-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8l/df7brwpx0vj8qwl22c82bj5w0000gn/T/b7ada75ec03746a6a7a778cc1a16c0ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 981 COLUMNS
At line 11998 RHS
At line 12975 BOUNDS
At line 12976 ENDATA
Problem MODEL has 976 rows, 3672 columns and 7344 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 0.4 - largest nonzero change 7.0735384e-06 ( 0.005696173%) - largest ze

### Extract and Display the Results

In [25]:
# Production Plan
print("\nProduction Plan:")
for v in prob.variables():
    if "Production" in v.name and v.varValue > 0:
        print(f"{v.name} = {v.varValue}")

# Transportation Plan
print("\nTransportation Plan:")
for v in prob.variables():
    if "Transport" in v.name and v.varValue > 0:
        print(f"{v.name} = {v.varValue}")

# Total Cost
print(f"\nTotal Operating Cost = ${value(prob.objective):,.2f}")


Production Plan:
Production_('KRCH',_'DA_Classic') = 1482446500000.0
Production_('KRCH',_'DA_Economy') = 1605090900000.0
Production_('KRCH',_'DA_Prem') = 1611169900000.0
Production_('KRCH',_'DM_Classic') = 1456893900000.0
Production_('KRCH',_'DM_Economy') = 101414590000.0
Production_('KRCH',_'DM_Prem') = 1609320700000.0
Production_('KRCH',_'DS_Classic') = 1636948500000.0
Production_('KRCH',_'DS_Economy') = 1585550800000.0
Production_('KRCH',_'DS_Prem') = 1450555200000.0
Production_('KRCH',_'MA_Classic') = 94317167000.0
Production_('KRCH',_'MA_Economy') = 103757980000.0
Production_('KRCH',_'MA_Prem') = 103210750000.0
Production_('KRCH',_'MM_Classic') = 98150625000.0
Production_('KRCH',_'MM_Economy') = 95798533000.0
Production_('KRCH',_'MM_Prem') = 97540484000.0
Production_('KRCH',_'MS_Classic') = 96991883000.0
Production_('KRCH',_'MS_Economy') = 98488498000.0
Production_('KRCH',_'MS_Prem') = 93786952000.0
Production_('PLLU',_'DM_Prem') = 37298613.0
Production_('TRME',_'DM_Prem') = 4016